In [1]:
import os

import numpy as np
import pandas as pd
import xml.etree.ElementTree as ET

### Konstruktikon:

In [2]:
def parse_sentences(xml_file):
    tree = ET.parse(xml_file)
    root = tree.getroot()

    construction_id = root.attrib.get('id')
    try:
        category, name = root.attrib.get('name').split(':')
    except ValueError:
        category = root.attrib.get('name')
        name = None

    sentences_data = []
    
    for sentence in root.findall('.//sentence'):
        if sentence.attrib.get('uid') is None:
            continue
        uid = sentence.attrib.get('uid')
        text = sentence.find('.//text').text.strip()
        contextleft = sentence.find('.//contextleft').text.strip()
        contextright = sentence.find('.//contextright').text.strip()
        
        sentences_data.append({
            'uid': uid,
            'text': text,
            'contextleft': contextleft,
            'contextright': contextright,
            'construction_id': int(construction_id),
            'category': category,
            'name': name,
        })

    return sentences_data

kee_list = []
xml_directory = '../data/constructicon/construction'

for filename in os.listdir(xml_directory):
    if filename.endswith('.xml'):
        xml_file = os.path.join(xml_directory, filename)
        data = parse_sentences(xml_file)
        if data:
            kee_list += data

sentences = pd.DataFrame.from_dict(kee_list)
sentences.set_index('uid', inplace=True)
sentences

,text,contextleft,contextright,construction_id,category,name
uid,,,,,,
57B9D1BEB68F44E7A51D1920F1EA6951E39469B5,Und dann ist da noch das generelle Problem mit...,"""""Wir haben die Gaza-Offensive nicht begonnen,...",Der ehemalige Ministerpräsident Hanija gehört ...,10,Negation,NEG_X_geschweige_denn_Y
D86B38CFA5D2458D9F3615EDD55C9308DF423DB0,»Ohne Europa sind viele Fragen nicht mehr seri...,Die Zeit nationaler Alleingänge sei längst vor...,"Die Frage ist: Kann es Europa gelingen, sich i...",10,Negation,NEG_X_geschweige_denn_Y
7B3FAB8B01ED862D3D3FA18531B0904BCF24DFF0,"Dies lässt sich damit begründen , dass vor und...","Wenn Alexander das Heer nicht geführt hätte, d...",Ganz anders ist die Beweislage für den ersten ...,10,Negation,NEG_X_geschweige_denn_Y
B003E526D0AA301DA46458D32A82CEEFD7ADADA5,"Dennoch hat Hamas es nicht geschafft , ihre mi...",DerCouncil of Foreign Relations schätzt das jä...,"Es ist in erster Linie die politische Führung,...",10,Negation,NEG_X_geschweige_denn_Y
C3FFF1CA55CA8FF0A806C33E14A0FB331582698E,"Abermillionen rings um die Welt , die das Spek...",Nichts hätte die Mär vom Niedergang der Superm...,"Doch wie schon die Wahl Obamas global war, ist...",10,Negation,NEG_X_geschweige_denn_Y
...,...,...,...,...,...,...
B5CB5D20EFEDAF65D460621CD3D554622B9EE1E9,""" "" Aber da wird geredet und geredet und , sob...",Zuerst hat er Flirtportale im Netz ausprobiert.,"Also geht er, der Computerfreak, lieber den al...",993,Eigenschaft,Intransitiv_passiv
32E879397C3BA674F547058D7CB2B781C55FE1AC,"Das erfüllt einen mit Stolz "" "" , sagte Neid u...","Das ist ja das Besondere, wenn man so ein Turn...",Das taten die Spielerinnen schon im legendären...,993,Eigenschaft,Intransitiv_passiv
F0F280AA87B32AF6C4122E5B9C7906A31E88E555,"Hier wird getankt , und erst in der Morgenfrüh...",Schließlich schafft es der Wagen wieder auf di...,Von dort geht es gleich ins Berchtesgadener La...,993,Eigenschaft,Intransitiv_passiv


In [3]:
constructions = sentences[['construction_id', 'category', 'name']].drop_duplicates()
constructions.set_index('construction_id', inplace=True)
constructions

,category,name
construction_id,,
10,Negation,NEG_X_geschweige_denn_Y
100,Äquativ_Plural,gleich_ADJ
1004,Superlativ_Klimax,ADJ1_ADJ1-er_NP
1006,Superlativ,PRÄP_ADJ-ster_NP
101,Äquativ,ADJ_wie_NP
...,...,...
98,Äquativ,so_ADJ_wie_XP
980,Quantifizierung,CARD_KATEGORIE_NP
985,Relation_zweistellig,Transitiv_aktiv


In [4]:
constructions['category'].value_counts()

category
Quantifizierung                     14
Intensivierung                      14
Kategorisierung                     13
Exklamativ                          13
Äquativ                             12
                                    ..
Reflexives_Partikelverb              1
Reflexiver_Weg                       1
Konkretisierung_exhaustiv            1
Konkretisierung_exemplifizierend     1
Korrelation_Affirmation              1
Name: count, Length: 181, dtype: int64

In [5]:
def parse_kee(xml_file):
    tree = ET.parse(xml_file)
    root = tree.getroot()

    construction_id = root.attrib.get('id')

    ke_data = []
    
    for ke in root.findall('.//ke'):
        ke_data.append({
            'ke_id': int(ke.attrib.get("id")),
            'ke_name': ke.attrib.get("name"),
            'ke_coretype': ke.attrib.get("coretype"),
            'ke_attributes': ke.attrib.get("attributes")
        })

    kee_data = []
    
    for kee in root.findall('.//kee'):
        for ke in ke_data:
            kee_data.append({
                'kee_id': int(kee.attrib.get("id")),
                'kee_name': kee.attrib.get("name"),
                'construction_id': int(construction_id)
            } | ke)

    return kee_data

kee_list = []
xml_directory = '../data/constructicon/construction'

for filename in os.listdir(xml_directory):
    if filename.endswith('.xml'):
        xml_file = os.path.join(xml_directory, filename)
        data = parse_kee(xml_file)
        if data:
            kee_list += data

kees = pd.DataFrame.from_dict(kee_list)
kees.set_index('kee_id', inplace=True)
kees

,kee_name,construction_id,ke_id,ke_name,ke_coretype,ke_attributes
kee_id,,,,,,
1,geschweige_denn,10,1,Fokuskontext,CORE,
1,geschweige_denn,10,2,Erstes_Konjunkt,CORE,
1,geschweige_denn,10,3,Zweites_Konjunkt,CORE,[]
1,geschweige_denn,10,2350,Negator,CORE,
1,geschweige_denn,10,1,Fokuskontext,None,None
...,...,...,...,...,...,...
643,er,99,336,Dimension,CORE,
643,er,99,337,Bezugswert,CORE,
643,er,99,338,Quantifizierung,NONCORE,


In [6]:
kees['ke_name'].value_counts()

ke_name
Verglichenes        108
Dimension            78
Bezugskontext        78
Bezugswert           77
Antezedens           64
                   ... 
Zugeordnetes          1
Von_X_Geäußertes      1
Sprecher_X            1
Sprecher              1
Direktiv              1
Name: count, Length: 247, dtype: int64

In [7]:
kees['kee_name'].value_counts()

kee_name
(_)                      75
Klammern                 52
von                      41
sein                     38
und                      36
                         ..
par_excellence            1
wie_es_im_Buche_steht     1
zu_Tode                   1
wie_viel_Uhr              1
nicht_un-                 1
Name: count, Length: 372, dtype: int64

Die KEEs sind markiert wie folgt:

```
            <annotationset>
            ...
                <layer name="KEE-Negation:NEG_X_geschweige_denn_Y" descriptor="KEE-Negation:NEG_X_geschweige_denn_Y">
                    <label start="72" end="87" name="geschweige_denn" refid="GC_KEE:1" itype="null" groupid="0"></label>
                </layer>
            ...
            </annotationset>
```

In [8]:
# TODO Datensatz erstellen gemäß csv bei Pseudowords: KE maskieren, KEE desambiguieren

In [18]:
def parse_masked_sentences(xml_file):
    tree = ET.parse(xml_file)
    root = tree.getroot()
    
    sentences_data = []
    
    constr_id = int(root.attrib.get('id'))
    
    for sentence in root.findall('.//sentence'):
        if sentence.attrib.get('uid') is None:
            continue
        
        uid = sentence.attrib.get('uid')
        text = sentence.find('.//text').text.strip()
        
        kees = []
        kees_idx = []
        kes = []
        kes_idx = []
        
        # Loop through annotations:
        for layer in sentence.findall('.//layer'):
            layer_name = layer.attrib.get('name')
            
            # Get the KEE annotations:
            if "KE-" in layer_name or "KEE-" in layer_name:
                # Loop over all KEEs or KEs:
                for label in layer.findall('.//label'):
                    start = int(label.attrib.get('start'))
                    end = int(label.attrib.get('end'))
                    if "KEE-" in layer_name:
                        kees.append(text[start:end])  # Read the KEE
                        kees_idx.append((start, end))  # Log the position of the KEE
                    else:
                        kes.append(text[start:end])  # Read the KE
                        kes_idx.append((start, end))  # Log the position of the KE
        
        sentences_data.append({
            'uid': uid,
            'constr_id': constr_id,
            'text': text,
            'kees': kees,
            'kees_idx': kees_idx,
            'kes': kes,
            'kes_idx': kes_idx
        })
    return sentences_data
    
sentence_list = []
xml_directory = '../data/constructicon/construction'

for filename in os.listdir(xml_directory):
    if filename.endswith('.xml'):
        xml_file = os.path.join(xml_directory, filename)
        data = parse_masked_sentences(xml_file)
        if data:
            sentence_list += data

sentences = pd.DataFrame.from_dict(sentence_list)
sentences.set_index('uid', inplace=True)
sentences    

,constr_id,text,kees,kees_idx,kes,kes_idx
uid,,,,,,
57B9D1BEB68F44E7A51D1920F1EA6951E39469B5,10,Und dann ist da noch das generelle Problem mit...,[geschweige denn],"[(128, 143)]","[dass, nicht, jeder Sprecher und Führer , der ...","[(55, 59), (60, 65), (66, 110), (111, 125), (1..."
D86B38CFA5D2458D9F3615EDD55C9308DF423DB0,10,»Ohne Europa sind viele Fragen nicht mehr seri...,[geschweige denn],"[(72, 87)]","[»Ohne Europa sind viele Fragen, nicht, mehr s...","[(0, 30), (31, 36), (37, 48), (49, 69), (88, 97)]"
7B3FAB8B01ED862D3D3FA18531B0904BCF24DFF0,10,"Dies lässt sich damit begründen , dass vor und...",[geschweige denn],"[(126, 141)]",[dass vor und neben Alexander – soweit wir wis...,"[(34, 84), (85, 89), (90, 97), (98, 123), (142..."
B003E526D0AA301DA46458D32A82CEEFD7ADADA5,10,"Dennoch hat Hamas es nicht geschafft , ihre mi...",[geschweige denn],"[(83, 98)]","[Dennoch hat Hamas es, nicht, geschafft, ihre ...","[(0, 20), (21, 26), (27, 36), (39, 80), (99, 1..."
C3FFF1CA55CA8FF0A806C33E14A0FB331582698E,10,"Abermillionen rings um die Welt , die das Spek...",[geschweige denn],"[(183, 198)]","[Abermillionen rings um die Welt , die das Spe...","[(0, 92), (93, 98), (99, 180), (201, 246)]"
...,...,...,...,...,...,...
B5CB5D20EFEDAF65D460621CD3D554622B9EE1E9,993,""" "" Aber da wird geredet und geredet und , sob...",[],[],[wird geredet],"[(12, 24)]"
32E879397C3BA674F547058D7CB2B781C55FE1AC,993,"Das erfüllt einen mit Stolz "" "" , sagte Neid u...",[],[],[wird getanzt],"[(105, 117)]"
F0F280AA87B32AF6C4122E5B9C7906A31E88E555,993,"Hier wird getankt , und erst in der Morgenfrüh...",[],[],[wird getankt],"[(5, 17)]"
